In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [4]:
path = '../data/nparrs/'
savepath = '../data/'
images = glob.glob(path+'*.npy')
totalImages = 1800

In [5]:
def myGenerator(batch_size):
    while True:
        index_list = random.sample(range(1, totalImages), batch_size)
        alldata_x = []
        alldata_y = []
        
        for i in index_list:
            frame = images[i]
            frame = np.load(frame)
            
            tile_index = np.random.randint(0, 199)
            temp       = tile_index*totalImages+i
            b          = frame[tile_index][:, :, :]
            
            alldata_y.append(b)
            temp       = imresize(b, (12, 12))
            tempa      = imresize(temp, (192, 192))
            
            alldata_x.append(tempa)
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        #alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        #alldata_x = (alldata_x.astype(np.float32) - 127.5) / 127.5
        yield alldata_x, alldata_y

x = myGenerator(10)
xtrain, ytrain = next(x)
print('xtrain shape:',xtrain.shape)
print('ytrain shape:',ytrain.shape)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


xtrain shape: (10, 192, 192, 3)
ytrain shape: (10, 192, 192, 3)


In [7]:
class CGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 384
        self.img_cols = 384
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(384, 384, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        input_size = (384,384,3)
        inputs = Input(input_size)

        conv1_a = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1_a = LeakyReLU(alpha=0.2)(conv1_a)
        conv1_a = BatchNormalization(momentum=0.8)(conv1_a)

        conv1_a = Conv2D(16, 3, padding = 'valid', kernel_initializer = 'he_normal')(conv1_a)
        conv1_a = LeakyReLU(alpha=0.2)(conv1_a)
        conv1_a = BatchNormalization(momentum=0.8)(conv1_a)

        pool1_a = Conv2D(16, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv1_a)
        pool1_a = LeakyReLU(alpha=0.2)(pool1_a)
        pool1_a = BatchNormalization(momentum=0.8)(pool1_a)
        pool1_a = ZeroPadding2D(padding=(1, 1))(pool1_a)



        conv1 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1_a)
        conv1 = LeakyReLU(alpha=0.2)(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)

        conv1 = Conv2D(32, 3, padding = 'valid', kernel_initializer = 'he_normal')(conv1)
        conv1 = LeakyReLU(alpha=0.2)(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)

        pool1 = Conv2D(32, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv1)
        pool1 = LeakyReLU(alpha=0.2)(pool1)
        pool1 = BatchNormalization(momentum=0.8)(pool1)



        conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
        conv2 = LeakyReLU(alpha=0.2)(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv2 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv2 = LeakyReLU(alpha=0.2)(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        pool2 = Conv2D(64, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv2)
        pool2 = LeakyReLU(alpha=0.2)(pool2)
        pool2 = BatchNormalization(momentum=0.8)(pool2)



        conv2_a = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
        conv2_a = LeakyReLU(alpha=0.2)(conv2_a)
        conv2_a = BatchNormalization(momentum=0.8)(conv2_a)

        conv2_a = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2_a)
        conv2_a = LeakyReLU(alpha=0.2)(conv2_a)
        conv2_a = BatchNormalization(momentum=0.8)(conv2_a)

        pool3   = Conv2D(64, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv2_a)
        pool3   = LeakyReLU(alpha=0.2)(pool3)
        pool3   = BatchNormalization(momentum=0.8)(pool3)



        conv3 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
        conv3 = LeakyReLU(alpha=0.2)(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        conv3 = Conv2D(1, 3, padding = 'valid', kernel_initializer = 'he_normal')(conv3)
        conv3 = LeakyReLU(alpha=0.2)(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)


        conv3_pool = Conv2D(1, 3, subsample=(2, 2), kernel_initializer = 'he_normal')(conv3)
        conv3_pool = LeakyReLU(alpha=0.2)(conv3_pool)
        conv3_pool = BatchNormalization(momentum=0.8)(conv3_pool)

        # #Decoder

        conv3_unpool = UpSampling2D(size = (2,2))(conv3_pool)
        conv3_unpool = ZeroPadding2D(padding=(1, 1))(conv3_unpool)

        conv3_unpool = Conv2D(1, 3, padding = 'same', kernel_initializer = 'he_normal')(conv3_unpool)
        conv3_unpool = LeakyReLU(alpha=0.2)(conv3_unpool)
        conv3_unpool = BatchNormalization(momentum=0.8)(conv3_unpool)


        up1    = UpSampling2D(size = (2,2))(conv3_unpool)
        #up1    = UpSampling2D(size = (2,2))(conv3)
        up1    = ZeroPadding2D(padding=(3, 3))(up1)


        conv4 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(up1)
        conv4 = LeakyReLU(alpha=0.2)(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        merge1 = concatenate([conv4, conv2_a], axis = 3)

        conv5 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(merge1)
        conv5 = LeakyReLU(alpha=0.2)(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        conv5 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv5)
        conv5 = LeakyReLU(alpha=0.2)(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)


        up2    = UpSampling2D(size = (2,2))(conv5)
        up2    = ZeroPadding2D(padding=(1, 1))(up2)

        conv6 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(up2)
        conv6 = LeakyReLU(alpha=0.2)(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        merge2 = concatenate([conv2, conv6], axis = 3)

        conv7 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(merge2)
        conv7 = LeakyReLU(alpha=0.2)(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        conv8 = Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)
        conv8 = LeakyReLU(alpha=0.2)(conv8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)

        up3    = UpSampling2D(size = (2,2))(conv8)
        up3    = ZeroPadding2D(padding=(1, 1))(up3)

        merge3 = concatenate([conv1, up3], axis = 3)
        merge3 = ZeroPadding2D(padding=(1, 1))(merge3)

        conv9 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(merge3)
        conv9 = LeakyReLU(alpha=0.2)(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)

        conv10 = Conv2D(32, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = LeakyReLU(alpha=0.2)(conv10)
        conv10 = BatchNormalization(momentum=0.8)(conv10)


        up4    = UpSampling2D(size = (2,2))(conv10)

        conv11 = Conv2D(16, 3, padding = 'valid', kernel_initializer = 'he_normal')(up4)
        conv11 = LeakyReLU(alpha=0.2)(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)

        merge4 = concatenate([conv1_a, conv11], axis = 3)
        merge4 = ZeroPadding2D(padding=(1, 1))(merge4)

        conv12 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(merge4)
        conv12 = LeakyReLU(alpha=0.2)(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)

        conv13 = Conv2D(16, 3, padding = 'same', kernel_initializer = 'he_normal')(conv12)
        conv13 = LeakyReLU(alpha=0.2)(conv13)
        conv13 = BatchNormalization(momentum=0.8)(conv13)


        conv14 = Conv2D(3, 1, padding = 'same', kernel_initializer = 'he_normal', activation='tanh')(conv13)
        
        model  = Model(input = inputs, output = conv14)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, X_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')
            
    def build_discriminator(self):
        img   = Input(shape=(384, 384, 3))
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3), input_shape=(192, 192, 3)))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64,  (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (6, 6),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        #model.add(Dropout(0.25))
        model.add(Conv2D(64, (3, 3),  strides=(2, 2)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        
        model.add(Dense(100))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(32))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1))
        
        output    = model(img)
        
        model3 = Model(img, output)
        return model3

    def train(self, epochs, batch_size=128, sample_interval=50):
        random.seed(10)
        
        # Load the dataset
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            
            # Adversarial ground truths
            valid = np.ones((batch_size, 1))
            fake  = np.zeros((batch_size, 1))
            
            # ---------------------
            #  Train Discriminator
            # ---------------------
            
            # Generate a half batch of new images
            gen_imgs = self.generator.predict(X_train)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(X_train, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator
            g_loss = self.combined.train_on_batch(X_train, [X_train, valid])

            # Plot the progress
            print ("%d [D loss: %f, mean_acc: %.2f%% real_acc: %.2f%% fake_acc: %.2f%%] [G loss: %f, mse: %f]" % (epoch, d_loss[0], 100*d_loss[1], 100*d_loss_real[1], 100*d_loss_fake[1], g_loss[0], g_loss[1]))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)
                self.generator.save_weights(savepath+'weights/generator_weights_'+str(epoch)+'.h5')
                self.discriminator.save_weights(savepath+'weights/discriminator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)

In [ ]:
# cgan = CGAN()
# cgan.build_autoencoder()
# cgan.train_generator_autoencoder(1000000, 10, 100)

In [ ]:
cgan.train(10000, batch_size=10, sample_interval=100)

In [8]:
cgan2 = CGAN()
#cgan2.generator.load_weights('../data/weights/generator_weights_3000.h5')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:67: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:81: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:95: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:110: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, 3, kernel_initializer="he_normal", strides=(2, 2))`
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 384, 384, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 384, 384, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 384, 384, 16) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 384, 384, 16) 64          leaky_re_lu_1[0][0]              
__________________________________________________________________________________________________
conv2d_2 (

In [43]:
path = '../data/sources/'
def myGenerator(batch_size):
    global tdx
    while True:
        index_list = random.sample(range(1, totalImages), batch_size)
        alldata_y = []
        for i in range(30, 120):#index_list:
            frame = np.load(path+'nparrs_384/frame'+str(i)+'.npy')
            alldata_y.append(frame[30])
        alldata_y = np.array(alldata_y)
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        yield alldata_y, alldata_y

In [44]:
savepath = '../data/sources/'
for i in range(1):
    x = myGenerator(90)
    X_train, ytest = next(x)

    gen_imgs         = cgan.generator.predict(X_train)

    # Rescale images 0 - 1
    temp     = (0.5 * gen_imgs + 0.5)*255
    X_train = (0.5 * X_train + 0.5)*255
    gen_imgs = temp.astype(int)
    
    idx = 0
    for j in gen_imgs:
        imsave(savepath+'prediction/tile_segment_image'+str(idx)+'.png', j)
        idx += 1
    idx = 0
    for j in X_train:
        imsave(savepath+'real/tile_segment_image_real'+str(idx)+'.png', j)
        idx += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [16]:
import time
a = []
for i in range(10):
    stime = time.time()
    pred = cgan2.generator.predict(xtrain)
    etime = time.time()
    tp = etime-stime
    a.append(tp)
    print(tp)
print(np.mean(a))

4.467476844787598
1.785005807876587
1.7195923328399658
1.7406260967254639
1.7078578472137451
1.7259161472320557
1.7722742557525635
1.7466368675231934
1.686011552810669
1.7431378364562988
2.009453558921814
